# 9-6 RNN을 이용한 텍스트 생성(Text Generation using RNN)
Many-to-one 구조

## 1. 데이터에 대한 이해와 전처리

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [3]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [4]:
t = Tokenizer()
t.fit_on_texts([text])
vocab_size = len(t.word_index) + 1 # 배열의 인덱스는 0부터 시작
print('단어 집합의 크기: %d' % vocab_size)
print(t.word_index)

단어 집합의 크기: 12
{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [5]:
sequences = list()

for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))
print(sequences) # 모델이 문맥을 학습할 수 있도록 재구성한 데이터

학습에 사용할 샘플의 개수: 11
[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [6]:
# 길이가 가장 긴 샘플의 길이 찾기
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이: {}'.format(max_len))

샘플의 최대 길이: 6


In [7]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [8]:
# 각 샘플의 마지막 단어 = 레이블
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]
print(X)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [9]:
# label 원-핫 인코딩
y = to_categorical(y, num_classes=vocab_size)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## 2. 모델 설계

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [12]:
!pip install numpy==1.19.5

In [11]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) # input_length는 레이블 제외
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.4919 - accuracy: 0.0000e+00
Epoch 2/200
1/1 - 0s - loss: 2.4781 - accuracy: 0.0000e+00
Epoch 3/200
1/1 - 0s - loss: 2.4647 - accuracy: 0.0000e+00
Epoch 4/200
1/1 - 0s - loss: 2.4515 - accuracy: 0.0000e+00
Epoch 5/200
1/1 - 0s - loss: 2.4384 - accuracy: 0.2727
Epoch 6/200
1/1 - 0s - loss: 2.4254 - accuracy: 0.2727
Epoch 7/200
1/1 - 0s - loss: 2.4124 - accuracy: 0.2727
Epoch 8/200
1/1 - 0s - loss: 2.3991 - accuracy: 0.4545
Epoch 9/200
1/1 - 0s - loss: 2.3856 - accuracy: 0.4545
Epoch 10/200
1/1 - 0s - loss: 2.3717 - accuracy: 0.4545
Epoch 11/200
1/1 - 0s - loss: 2.3573 - accuracy: 0.3636
Epoch 12/200
1/1 - 0s - loss: 2.3424 - accuracy: 0.3636
Epoch 13/200
1/1 - 0s - loss: 2.3268 - accuracy: 0.3636
Epoch 14/200
1/1 - 0s - loss: 2.3104 - accuracy: 0.3636
Epoch 15/200
1/1 - 0s - loss: 2.2932 - accuracy: 0.3636
Epoch 16/200
1/1 - 0s - loss: 2.2752 - accuracy: 0.3636
Epoch 17/200
1/1 - 0s - loss: 2.2563 - accuracy: 0.3636
Epoch 18/200
1/1 - 0s - loss: 2.2365 - ac

Epoch 147/200
1/1 - 0s - loss: 0.2402 - accuracy: 1.0000
Epoch 148/200
1/1 - 0s - loss: 0.2353 - accuracy: 1.0000
Epoch 149/200
1/1 - 0s - loss: 0.2305 - accuracy: 1.0000
Epoch 150/200
1/1 - 0s - loss: 0.2259 - accuracy: 1.0000
Epoch 151/200
1/1 - 0s - loss: 0.2214 - accuracy: 1.0000
Epoch 152/200
1/1 - 0s - loss: 0.2169 - accuracy: 1.0000
Epoch 153/200
1/1 - 0s - loss: 0.2126 - accuracy: 1.0000
Epoch 154/200
1/1 - 0s - loss: 0.2083 - accuracy: 1.0000
Epoch 155/200
1/1 - 0s - loss: 0.2042 - accuracy: 1.0000
Epoch 156/200
1/1 - 0s - loss: 0.2001 - accuracy: 1.0000
Epoch 157/200
1/1 - 0s - loss: 0.1962 - accuracy: 1.0000
Epoch 158/200
1/1 - 0s - loss: 0.1923 - accuracy: 1.0000
Epoch 159/200
1/1 - 0s - loss: 0.1885 - accuracy: 1.0000
Epoch 160/200
1/1 - 0s - loss: 0.1848 - accuracy: 1.0000
Epoch 161/200
1/1 - 0s - loss: 0.1812 - accuracy: 1.0000
Epoch 162/200
1/1 - 0s - loss: 0.1776 - accuracy: 1.0000
Epoch 163/200
1/1 - 0s - loss: 0.1742 - accuracy: 1.0000
Epoch 164/200
1/1 - 0s - loss: 

In [2]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복횟수
    init_word = current_word
    sentence = ''
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩

1.19.5
